In [1]:
from fastai.vision.all import *

In [2]:
defaults.device = torch.device('cpu')
path = Path("/home/alarm/projects/birbcam/imgs/")

In [3]:
files = get_image_files(path/"training")

In [4]:
len(files)

3567

In [5]:
def labeler(x):
    x = str(x)
    x = x.split('/')[-1]
    x = x.split('_')
    x = x[:-1]
    if 'none' in x:
        x.remove('none')
    return ','.join(x)

df = pd.DataFrame({'fname': [str(f).split('/')[-1] for f in files]})
df['labels'] = df['fname'].apply(labeler)

In [6]:
df = df[df["labels"] != ''].reset_index(drop=True)
#df = df[df["labels"] != 'magpie'].reset_index(drop=True)

In [7]:
df['labels'].unique()

array(['sparrow', 'chickadee', 'squirrel', 'magpie'], dtype=object)

In [8]:
df.head()

,fname,labels
0,sparrow_2020-12-02T10:20:06.jpg,sparrow
1,sparrow_2020-12-02T12:54:04.jpg,sparrow
2,sparrow_2020-11-30T09:24:00.jpg,sparrow
3,chickadee_2020-12-26T13:08:05.jpg,chickadee
4,chickadee_2020-12-19T10:19:23.jpg,chickadee


In [9]:
dls = ImageDataLoaders.from_df(df, path, folder='training', label_delim=',', item_tfms=Resize(224), batch_tfms=aug_transforms(size=224))

Could not do one pass in your dataloader, there is something wrong in it
